In [161]:
import pandas as pd
import numpy as np
from random import randint


def generate_location_data(patient_count: int, location_count: int) -> pd.DataFrame:
    row_count = patient_count * 10
    patient_ids = np.random.randint(1, patient_count + 1, size=row_count)
    locations = np.array([str(chr(randint(65, 65 + patient_count - 1))) for i in range(row_count)])
    df = pd.DataFrame()
    df['ptid'] = pd.Series(patient_ids)
    df['loc'] = pd.Series(locations)
    return df.sort_values(by='ptid').reset_index(drop=True)

def get_transfers(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df['from_loc'] = df['loc']
    df['to_ptid'] = pd.Series(np.roll(df['ptid'].values, -1))
    df['to_loc'] = pd.Series(np.roll(df['loc'].values, -1))
    df = df[df['ptid'] == df['to_ptid']]
    return df.drop(columns=['loc', 'to_ptid'])

In [162]:
locations = generate_location_data(20, 5)
transfers = get_transfers(locations)

In [171]:
tx = transfers.copy()
tx['transfer'] = tx['from_loc'] + ' -> ' + tx['to_loc']
tx = tx.drop(columns=['from_loc', 'to_loc'])
tx_sorted = tx.sort_values(by=['ptid', 'transfer']).reset_index(drop=True)
tx_sorted['count'] = pd.Series(np.ones(shape=len(tx_sorted)))
tx_grouped = tx_sorted.groupby(by=['ptid', 'transfer']).count()

In [173]:
tx_grouped.unstack(1).fillna(0)

count                                                          \
transfer A -> A A -> C A -> E A -> F A -> I A -> J A -> O A -> R B -> B   
ptid                                                                      
1           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2           1.0    0.0    0.0    0.0    1.0    0.0    0.0    0.0    0.0   
3           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4           1.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0   
5           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6           0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
7           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0   
8           0.0    0.0    0.0    1.0    0.0    0.0    0.0    0.0    0.0   
9           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0   
10          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
11          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
12          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
13          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
14          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
15          1.0    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0   
16          0.0    0.0    1.0    0.0    0.0    1.0    0.0    0.0    0.0   
17          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
18          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
19          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
20          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                 ...                                                     \
transfer B -> C  ...   S -> N S -> R S -> S T -> D T -> G T -> J T -> K   
ptid             ...                                                      
1           0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2           0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3           0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4           0.0  ...      0.0    0.0    0.0    1.0    0.0    0.0    0.0   
5           0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6           0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
7           0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
8           0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9           0.0  ...      1.0    0.0    0.0    0.0    0.0    0.0    0.0   
10          0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
11          0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    1.0   
12          0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
13          0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
14          0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
15          0.0  ...      0.0    1.0    0.0    0.0    0.0    1.0    0.0   
16          1.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
17          0.0  ...      1.0    0.0    1.0    0.0    0.0    0.0    0.0   
18          0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   
19          0.0  ...      0.0    0.0    0.0    0.0    1.0    0.0    0.0   
20          0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                               
transfer T -> L T -> P T -> R  
ptid                           
1           0.0    0.0    0.0  
2           0.0    0.0    0.0  
3           0.0    1.0    0.0  
4           0.0    0.0    0.0  
5           0.0    0.0    0.0  
6           0.0    0.0    0.0  
7           0.0    1.0    0.0  
8           0.0    0.0    0.0  
9           0.0    0.0    0.0  
10          0.0    0.0    0.0  
11          0.0    0.0    0.0  
12          0.0    0.0    0.0  
13          0.0    0.0    0.0  
14          0.0    1.0    0.0  
15          0.